In [1]:
import tensorflow as tf
import tensorflow.keras.layers
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Dense, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

from sklearn.utils import shuffle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# # 혹시나 사용할 수 있는 가중이동평균 함수 지정 - 보수적 판단을 위해 사용 X

# def weighted_mean(weight_array) :
#   def inner(x) :
#     return (weight_array * x).mean()
#   return inner

# weights = np.arange(1,6)
# wma5 = df['close'].rolling(5).apply(lambda prices:np.dot(prices, weights)/weights.sum(), raw=True)
# ma5 = df['close'].rolling(5).mean()

# print(wma5, ma5)

In [4]:
def mov5(row_name) :
  return df[row_name].rolling(5).mean() 

def updown(row_name) :
  return df[row_name] - df[row_name].shift(1)

def var00(row_name) :
  return df[row_name].rolling(5).var()

In [5]:
df = pd.read_csv("/content/gdrive/My Drive/AAPL_dataset02.csv")
df.columns = ['date', 'ALPHA','BETA']
df2 = pd.read_csv("/content/gdrive/My Drive/AAPL.csv")
df2.columns = ['date', 'open', 'high', 'low', 'close', 'adj', 'volume', 'web', 'news', 'youtube', 'image', 'gold']
df2 = df2[['date','close']]
df2['close'] = df2['close'].rolling(5).mean()

df = pd.merge(df, df2, on= 'date', how='left')

# 모델 지정
row_names = ['ALPHA','BETA']

# for문을 이용해 각 모델 별 현재수준 / 변동값 / 분산 컬럼 계산해서 컬럼 생성하기

for i in range(len(row_names)) :
  mov5_name = row_names[i] + "_mov5"
  df[mov5_name] = mov5(row_names[i])

  updown_name = row_names[i] + "_updown"
  df[updown_name] = updown(row_names[i])

  var00_name = row_names[i] + "_var00"
  df[var00_name] = var00(row_names[i])

df = df.dropna(axis=0)

# 날짜 변수 저장
df2 = df['date'].values

# 종가 변수 저장
df3 = df['close'].values

# 백분위수 구하는 함수( quantile )를 변수로 지정

movquan_01 = df[mov5_name].quantile(q=0.9, interpolation= 'nearest')
movquan_02 = df[mov5_name].quantile(q=0.7, interpolation= 'nearest')
movquan_03 = df[mov5_name].quantile(q=0.4, interpolation= 'nearest')
varquan_04 = df[var00_name].quantile(q=0.5, interpolation='nearest')
varquan_05 = df[var00_name].quantile(q=0.75, interpolation='nearest')

# 모델 지정을 위한 for문 

for i in range(len(row_names)) :         

  # 모델 가져오기

  mov5_name = row_names[i] + "_mov5"
  updown_name = row_names[i] + "_updown"
  var00_name = row_names[i] + "_var00"
  
  # 최근 5일간 각 모델의 판단

  list1 = [-5,-4,-3,-2,-1]
  print("\n", row_names[i],"의 판단입니다.\n")

  for a in list1 :
    
    if df[mov5_name].values[a] >= movquan_01 :                                                                                                    # 현재수준 90% 이상
      print(df2[a],'기준 \'적극 투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')

    elif df[mov5_name].values[a] >= movquan_02 and df[updown_name].values[a] >= 0 :                                                               # 현재 수준 90% 미만 70% 이상 / 변동값 증가
      print(df2[a] ,'기준 \'투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')
    
    elif df[mov5_name].values[a] >= movquan_02 and df[updown_name].values[a] < 0 :                                                                # 현재 수준 90% 미만 70% 이상 / 변동값 감소
      print(df2[a] ,'기준 \'보수적 투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] > 0 and df[var00_name].values[a] > varquan_04 :         # 현재 수준 70% 미만 40% 이상 / 변동값 증가 / 분산 50% 초과
      print(df2[a] ,'기준 \'투자 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] <= 0 and df[var00_name].values[a] > varquan_04 :        # 현재 수준 70% 미만 40% 이상 / 변동값 감소 / 분산 50% 초과
      print(df2[a] ,'기준 \'투자 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] <= 0 and df[var00_name].values[a] < varquan_04 :        # 현재 수준 70% 미만 40% 이상 / 변동값 감소 / 분산 50% 미만
      print(df2[a] ,'기준 \'적정 시기가 아님\'을 추천하였습니다.')

    elif movquan_03 > df[mov5_name].values[a] and df[updown_name].values[a] > 0 and df[var00_name].values[a] > varquan_05 :                       # 현재 수준 40% 미만 / 변동값 증가 / 분산 75% 초과
      print(df2[a] ,'기준 \'오를 가능성이 있으니 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_03 > df[mov5_name].values[a] and df[updown_name].values[a] <= 0 :                                                                # 현재 수준 40% 미만 / 변동값 감소 
      print(df2[a] ,'기준 \'투자를 권하지 않음\'을 추천하였습니다.')

    else :                                                         
      print(df2[a] ,'기준 \'투자를 권하지 않음\'을 추천하였습니다.')  



 ALPHA 의 판단입니다.

20201030 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 113.41 입니다.
20201102 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 112.15 입니다.
20201103 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 110.92 입니다.
20201104 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 111.67 입니다.
20201105 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 112.41 입니다.

 BETA 의 판단입니다.

20201030 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 113.41 입니다.
20201102 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 112.15 입니다.
20201103 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 110.92 입니다.
20201104 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 111.67 입니다.
20201105 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 112.41 입니다.


In [6]:
df = pd.read_csv("/content/gdrive/My Drive/AMZN_dataset02.csv")
df.columns = ['date', 'ALPHA','BETA']
df2 = pd.read_csv("/content/gdrive/My Drive/AMZN.csv")
df2.columns = ['date', 'open', 'high', 'low', 'close', 'adj', 'volume', 'web', 'news', 'youtube', 'image', 'gold']
df2 = df2[['date','close']]
df2['close'] = df2['close'].rolling(5).mean()

df = pd.merge(df, df2, on= 'date', how='left')

# 모델 지정
row_names = ['ALPHA','BETA']

# for문을 이용해 각 모델 별 현재수준 / 변동값 / 분산 컬럼 계산해서 컬럼 생성하기

for i in range(len(row_names)) :
  mov5_name = row_names[i] + "_mov5"
  df[mov5_name] = mov5(row_names[i])

  updown_name = row_names[i] + "_updown"
  df[updown_name] = updown(row_names[i])

  var00_name = row_names[i] + "_var00"
  df[var00_name] = var00(row_names[i])

df = df.dropna(axis=0)

# 날짜 변수 저장
df2 = df['date'].values

# 종가 변수 저장
df3 = df['close'].values

# 백분위수 구하는 함수( quantile )를 변수로 지정

movquan_01 = df[mov5_name].quantile(q=0.9, interpolation= 'nearest')
movquan_02 = df[mov5_name].quantile(q=0.7, interpolation= 'nearest')
movquan_03 = df[mov5_name].quantile(q=0.4, interpolation= 'nearest')
varquan_04 = df[var00_name].quantile(q=0.5, interpolation='nearest')
varquan_05 = df[var00_name].quantile(q=0.75, interpolation='nearest')

# 모델 지정을 위한 for문 

for i in range(len(row_names)) :         

  # 모델 가져오기

  mov5_name = row_names[i] + "_mov5"
  updown_name = row_names[i] + "_updown"
  var00_name = row_names[i] + "_var00"
  
  # 최근 5일간 각 모델의 판단

  list1 = [-5,-4,-3,-2,-1]
  print("\n", row_names[i],"의 판단입니다.\n")

  for a in list1 :
    
    if df[mov5_name].values[a] >= movquan_01 :                                                                                                    # 현재수준 90% 이상
      print(df2[a],'기준 \'적극 투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')

    elif df[mov5_name].values[a] >= movquan_02 and df[updown_name].values[a] >= 0 :                                                               # 현재 수준 90% 미만 70% 이상 / 변동값 증가
      print(df2[a] ,'기준 \'투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')
    
    elif df[mov5_name].values[a] >= movquan_02 and df[updown_name].values[a] < 0 :                                                                # 현재 수준 90% 미만 70% 이상 / 변동값 감소
      print(df2[a] ,'기준 \'보수적 투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] > 0 and df[var00_name].values[a] > varquan_04 :         # 현재 수준 70% 미만 40% 이상 / 변동값 증가 / 분산 50% 초과
      print(df2[a] ,'기준 \'투자 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] <= 0 and df[var00_name].values[a] > varquan_04 :        # 현재 수준 70% 미만 40% 이상 / 변동값 감소 / 분산 50% 초과
      print(df2[a] ,'기준 \'투자 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] <= 0 and df[var00_name].values[a] < varquan_04 :        # 현재 수준 70% 미만 40% 이상 / 변동값 감소 / 분산 50% 미만
      print(df2[a] ,'기준 \'적정 시기가 아님\'을 추천하였습니다.')

    elif movquan_03 > df[mov5_name].values[a] and df[updown_name].values[a] > 0 and df[var00_name].values[a] > varquan_05 :                       # 현재 수준 40% 미만 / 변동값 증가 / 분산 75% 초과
      print(df2[a] ,'기준 \'오를 가능성이 있으니 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_03 > df[mov5_name].values[a] and df[updown_name].values[a] <= 0 :                                                                # 현재 수준 40% 미만 / 변동값 감소 
      print(df2[a] ,'기준 \'투자를 권하지 않음\'을 추천하였습니다.')

    else :                                                         
      print(df2[a] ,'기준 \'투자를 권하지 않음\'을 추천하였습니다.')  



 ALPHA 의 판단입니다.

20201030 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 3180.66 입니다.
20201102 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 3140.15 입니다.
20201103 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 3092.57 입니다.
20201104 기준 '보수적 투자 권장'을 추천하였습니다. 적정 매수 가격은 3108.24 입니다.
20201105 기준 '투자 권장'을 추천하였습니다. 적정 매수 가격은 3130.44 입니다.

 BETA 의 판단입니다.

20201030 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 3180.66 입니다.
20201102 기준 '보수적 투자 권장'을 추천하였습니다. 적정 매수 가격은 3140.15 입니다.
20201103 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 2999.79 입니다.
20201104 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 3014.99 입니다.
20201105 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 3036.53 입니다.


In [7]:
df = pd.read_csv("/content/gdrive/My Drive/AMD_dataset02.csv")
df.columns = ['date', 'ALPHA','BETA']
df2 = pd.read_csv("/content/gdrive/My Drive/AMD.csv")
df2.columns = ['date', 'open', 'high', 'low', 'close', 'adj', 'volume', 'web', 'news', 'youtube', 'image', 'gold']
df2 = df2[['date','close']]
df2['close'] = df2['close'].rolling(5).mean()

df = pd.merge(df, df2, on= 'date', how='left')

# 모델 지정
row_names = ['ALPHA','BETA']

# for문을 이용해 각 모델 별 현재수준 / 변동값 / 분산 컬럼 계산해서 컬럼 생성하기

for i in range(len(row_names)) :
  mov5_name = row_names[i] + "_mov5"
  df[mov5_name] = mov5(row_names[i])

  updown_name = row_names[i] + "_updown"
  df[updown_name] = updown(row_names[i])

  var00_name = row_names[i] + "_var00"
  df[var00_name] = var00(row_names[i])

df = df.dropna(axis=0)

# 날짜 변수 저장
df2 = df['date'].values

# 종가 변수 저장
df3 = df['close'].values

# 백분위수 구하는 함수( quantile )를 변수로 지정

movquan_01 = df[mov5_name].quantile(q=0.9, interpolation= 'nearest')
movquan_02 = df[mov5_name].quantile(q=0.7, interpolation= 'nearest')
movquan_03 = df[mov5_name].quantile(q=0.4, interpolation= 'nearest')
varquan_04 = df[var00_name].quantile(q=0.5, interpolation='nearest')
varquan_05 = df[var00_name].quantile(q=0.75, interpolation='nearest')

# 모델 지정을 위한 for문 

for i in range(len(row_names)) :         

  # 모델 가져오기

  mov5_name = row_names[i] + "_mov5"
  updown_name = row_names[i] + "_updown"
  var00_name = row_names[i] + "_var00"
  
  # 최근 5일간 각 모델의 판단

  list1 = [-5,-4,-3,-2,-1]
  print("\n", row_names[i],"의 판단입니다.\n")

  for a in list1 :
    
    if df[mov5_name].values[a] >= movquan_01 :                                                                                                    # 현재수준 90% 이상
      print(df2[a],'기준 \'적극 투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')

    elif df[mov5_name].values[a] >= movquan_02 and df[updown_name].values[a] >= 0 :                                                               # 현재 수준 90% 미만 70% 이상 / 변동값 증가
      print(df2[a] ,'기준 \'투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')
    
    elif df[mov5_name].values[a] >= movquan_02 and df[updown_name].values[a] < 0 :                                                                # 현재 수준 90% 미만 70% 이상 / 변동값 감소
      print(df2[a] ,'기준 \'보수적 투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] > 0 and df[var00_name].values[a] > varquan_04 :         # 현재 수준 70% 미만 40% 이상 / 변동값 증가 / 분산 50% 초과
      print(df2[a] ,'기준 \'투자 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] <= 0 and df[var00_name].values[a] > varquan_04 :        # 현재 수준 70% 미만 40% 이상 / 변동값 감소 / 분산 50% 초과
      print(df2[a] ,'기준 \'투자 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] <= 0 and df[var00_name].values[a] < varquan_04 :        # 현재 수준 70% 미만 40% 이상 / 변동값 감소 / 분산 50% 미만
      print(df2[a] ,'기준 \'적정 시기가 아님\'을 추천하였습니다.')

    elif movquan_03 > df[mov5_name].values[a] and df[updown_name].values[a] > 0 and df[var00_name].values[a] > varquan_05 :                       # 현재 수준 40% 미만 / 변동값 증가 / 분산 75% 초과
      print(df2[a] ,'기준 \'오를 가능성이 있으니 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_03 > df[mov5_name].values[a] and df[updown_name].values[a] <= 0 :                                                                # 현재 수준 40% 미만 / 변동값 감소 
      print(df2[a] ,'기준 \'투자를 권하지 않음\'을 추천하였습니다.')

    else :                                                         
      print(df2[a] ,'기준 \'투자를 권하지 않음\'을 추천하였습니다.')  



 ALPHA 의 판단입니다.

20201030 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 78.16 입니다.
20201102 기준 '투자 권장'을 추천하였습니다. 적정 매수 가격은 76.66 입니다.
20201103 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 73.91 입니다.
20201104 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 74.87 입니다.
20201105 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 75.84 입니다.

 BETA 의 판단입니다.

20201030 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 78.16 입니다.
20201102 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 76.66 입니다.
20201103 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 76.2 입니다.
20201104 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 77.19 입니다.
20201105 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 78.18 입니다.


In [8]:
df = pd.read_csv("/content/gdrive/My Drive/INTC_dataset02.csv")
df.columns = ['date', 'ALPHA','BETA']
df2 = pd.read_csv("/content/gdrive/My Drive/INTC.csv")
df2.columns = ['date', 'open', 'high', 'low', 'close', 'adj', 'volume', 'web', 'news', 'youtube', 'image', 'gold']
df2 = df2[['date','close']]
df2['close'] = df2['close'].rolling(5).mean()

df = pd.merge(df, df2, on= 'date', how='left')

# 모델 지정
row_names = ['ALPHA','BETA']

# for문을 이용해 각 모델 별 현재수준 / 변동값 / 분산 컬럼 계산해서 컬럼 생성하기

for i in range(len(row_names)) :
  mov5_name = row_names[i] + "_mov5"
  df[mov5_name] = mov5(row_names[i])

  updown_name = row_names[i] + "_updown"
  df[updown_name] = updown(row_names[i])

  var00_name = row_names[i] + "_var00"
  df[var00_name] = var00(row_names[i])

df = df.dropna(axis=0)

# 날짜 변수 저장
df2 = df['date'].values

# 종가 변수 저장
df3 = df['close'].values

# 백분위수 구하는 함수( quantile )를 변수로 지정

movquan_01 = df[mov5_name].quantile(q=0.9, interpolation= 'nearest')
movquan_02 = df[mov5_name].quantile(q=0.7, interpolation= 'nearest')
movquan_03 = df[mov5_name].quantile(q=0.4, interpolation= 'nearest')
varquan_04 = df[var00_name].quantile(q=0.5, interpolation='nearest')
varquan_05 = df[var00_name].quantile(q=0.75, interpolation='nearest')

# 모델 지정을 위한 for문 

for i in range(len(row_names)) :         

  # 모델 가져오기

  mov5_name = row_names[i] + "_mov5"
  updown_name = row_names[i] + "_updown"
  var00_name = row_names[i] + "_var00"
  
  # 최근 5일간 각 모델의 판단

  list1 = [-5,-4,-3,-2,-1]
  print("\n", row_names[i],"의 판단입니다.\n")

  for a in list1 :
    
    if df[mov5_name].values[a] >= movquan_01 :                                                                                                    # 현재수준 90% 이상
      print(df2[a],'기준 \'적극 투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')

    elif df[mov5_name].values[a] >= movquan_02 and df[updown_name].values[a] >= 0 :                                                               # 현재 수준 90% 미만 70% 이상 / 변동값 증가
      print(df2[a] ,'기준 \'투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')
    
    elif df[mov5_name].values[a] >= movquan_02 and df[updown_name].values[a] < 0 :                                                                # 현재 수준 90% 미만 70% 이상 / 변동값 감소
      print(df2[a] ,'기준 \'보수적 투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] > 0 and df[var00_name].values[a] > varquan_04 :         # 현재 수준 70% 미만 40% 이상 / 변동값 증가 / 분산 50% 초과
      print(df2[a] ,'기준 \'투자 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] <= 0 and df[var00_name].values[a] > varquan_04 :        # 현재 수준 70% 미만 40% 이상 / 변동값 감소 / 분산 50% 초과
      print(df2[a] ,'기준 \'투자 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] <= 0 and df[var00_name].values[a] < varquan_04 :        # 현재 수준 70% 미만 40% 이상 / 변동값 감소 / 분산 50% 미만
      print(df2[a] ,'기준 \'적정 시기가 아님\'을 추천하였습니다.')

    elif movquan_03 > df[mov5_name].values[a] and df[updown_name].values[a] > 0 and df[var00_name].values[a] > varquan_05 :                       # 현재 수준 40% 미만 / 변동값 증가 / 분산 75% 초과
      print(df2[a] ,'기준 \'오를 가능성이 있으니 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_03 > df[mov5_name].values[a] and df[updown_name].values[a] <= 0 :                                                                # 현재 수준 40% 미만 / 변동값 감소 
      print(df2[a] ,'기준 \'투자를 권하지 않음\'을 추천하였습니다.')

    else :                                                         
      print(df2[a] ,'기준 \'투자를 권하지 않음\'을 추천하였습니다.')  



 ALPHA 의 판단입니다.

20201030 기준 '투자를 권하지 않음'을 추천하였습니다.
20201102 기준 '투자를 권하지 않음'을 추천하였습니다.
20201103 기준 '투자를 권하지 않음'을 추천하였습니다.
20201104 기준 '투자를 권하지 않음'을 추천하였습니다.
20201105 기준 '투자를 권하지 않음'을 추천하였습니다.

 BETA 의 판단입니다.

20201030 기준 '투자를 권하지 않음'을 추천하였습니다.
20201102 기준 '투자를 권하지 않음'을 추천하였습니다.
20201103 기준 '투자를 권하지 않음'을 추천하였습니다.
20201104 기준 '투자를 권하지 않음'을 추천하였습니다.
20201105 기준 '투자를 권하지 않음'을 추천하였습니다.


In [9]:
df = pd.read_csv("/content/gdrive/My Drive/SBUX_dataset02.csv")
df.columns = ['date', 'ALPHA','BETA']
df2 = pd.read_csv("/content/gdrive/My Drive/SBUX_rev.2.csv")
df2.columns = ['date', 'open', 'high', 'low', 'close', 'adj', 'volume', 'web', 'news', 'youtube', 'image', 'gold']
df2 = df2[['date','close']]
df2['close'] = df2['close'].rolling(5).mean()

df = pd.merge(df, df2, on= 'date', how='left')

# 모델 지정
row_names = ['ALPHA','BETA']

# for문을 이용해 각 모델 별 현재수준 / 변동값 / 분산 컬럼 계산해서 컬럼 생성하기

for i in range(len(row_names)) :
  mov5_name = row_names[i] + "_mov5"
  df[mov5_name] = mov5(row_names[i])

  updown_name = row_names[i] + "_updown"
  df[updown_name] = updown(row_names[i])

  var00_name = row_names[i] + "_var00"
  df[var00_name] = var00(row_names[i])

df = df.dropna(axis=0)

# 날짜 변수 저장
df2 = df['date'].values

# 종가 변수 저장
df3 = df['close'].values

# 백분위수 구하는 함수( quantile )를 변수로 지정

movquan_01 = df[mov5_name].quantile(q=0.9, interpolation= 'nearest')
movquan_02 = df[mov5_name].quantile(q=0.7, interpolation= 'nearest')
movquan_03 = df[mov5_name].quantile(q=0.4, interpolation= 'nearest')
varquan_04 = df[var00_name].quantile(q=0.5, interpolation='nearest')
varquan_05 = df[var00_name].quantile(q=0.75, interpolation='nearest')

# 모델 지정을 위한 for문 

for i in range(len(row_names)) :         

  # 모델 가져오기

  mov5_name = row_names[i] + "_mov5"
  updown_name = row_names[i] + "_updown"
  var00_name = row_names[i] + "_var00"
  
  # 최근 5일간 각 모델의 판단

  list1 = [-5,-4,-3,-2,-1]
  print("\n", row_names[i],"의 판단입니다.\n")

  for a in list1 :
    
    if df[mov5_name].values[a] >= movquan_01 :                                                                                                    # 현재수준 90% 이상
      print(df2[a],'기준 \'적극 투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')

    elif df[mov5_name].values[a] >= movquan_02 and df[updown_name].values[a] >= 0 :                                                               # 현재 수준 90% 미만 70% 이상 / 변동값 증가
      print(df2[a] ,'기준 \'투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')
    
    elif df[mov5_name].values[a] >= movquan_02 and df[updown_name].values[a] < 0 :                                                                # 현재 수준 90% 미만 70% 이상 / 변동값 감소
      print(df2[a] ,'기준 \'보수적 투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] > 0 and df[var00_name].values[a] > varquan_04 :         # 현재 수준 70% 미만 40% 이상 / 변동값 증가 / 분산 50% 초과
      print(df2[a] ,'기준 \'투자 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] <= 0 and df[var00_name].values[a] > varquan_04 :        # 현재 수준 70% 미만 40% 이상 / 변동값 감소 / 분산 50% 초과
      print(df2[a] ,'기준 \'투자 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] <= 0 and df[var00_name].values[a] < varquan_04 :        # 현재 수준 70% 미만 40% 이상 / 변동값 감소 / 분산 50% 미만
      print(df2[a] ,'기준 \'적정 시기가 아님\'을 추천하였습니다.')

    elif movquan_03 > df[mov5_name].values[a] and df[updown_name].values[a] > 0 and df[var00_name].values[a] > varquan_05 :                       # 현재 수준 40% 미만 / 변동값 증가 / 분산 75% 초과
      print(df2[a] ,'기준 \'오를 가능성이 있으니 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_03 > df[mov5_name].values[a] and df[updown_name].values[a] <= 0 :                                                                # 현재 수준 40% 미만 / 변동값 감소 
      print(df2[a] ,'기준 \'투자를 권하지 않음\'을 추천하였습니다.')

    else :                                                         
      print(df2[a] ,'기준 \'투자를 권하지 않음\'을 추천하였습니다.')  



 ALPHA 의 판단입니다.

20201027 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 86.94 입니다.
20201028 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 86.73 입니다.
20201029 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 86.52 입니다.
20201030 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 85.78 입니다.
20201102 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 85.06 입니다.

 BETA 의 판단입니다.

20201027 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 86.94 입니다.
20201028 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 86.73 입니다.
20201029 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 86.52 입니다.
20201030 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 85.78 입니다.
20201102 기준 '투자 관망'을 추천하였습니다. 적정 매수 가격은 85.06 입니다.


In [10]:
df = pd.read_csv("/content/gdrive/My Drive/TSLA_dataset02.csv")
df.columns = ['date', 'ALPHA','BETA']
df2 = pd.read_csv("/content/gdrive/My Drive/TSLA.csv")
df2.columns = ['date', 'open', 'high', 'low', 'close', 'adj', 'volume', 'web', 'news', 'youtube', 'image', 'gold']
df2 = df2[['date','close']]
df2['close'] = df2['close'].rolling(5).mean()

df = pd.merge(df, df2, on= 'date', how='left')

# 모델 지정
row_names = ['ALPHA','BETA']

# for문을 이용해 각 모델 별 현재수준 / 변동값 / 분산 컬럼 계산해서 컬럼 생성하기

for i in range(len(row_names)) :
  mov5_name = row_names[i] + "_mov5"
  df[mov5_name] = mov5(row_names[i])

  updown_name = row_names[i] + "_updown"
  df[updown_name] = updown(row_names[i])

  var00_name = row_names[i] + "_var00"
  df[var00_name] = var00(row_names[i])

df = df.dropna(axis=0)

# 날짜 변수 저장
df2 = df['date'].values

# 종가 변수 저장
df3 = df['close'].values

# 백분위수 구하는 함수( quantile )를 변수로 지정

movquan_01 = df[mov5_name].quantile(q=0.9, interpolation= 'nearest')
movquan_02 = df[mov5_name].quantile(q=0.7, interpolation= 'nearest')
movquan_03 = df[mov5_name].quantile(q=0.4, interpolation= 'nearest')
varquan_04 = df[var00_name].quantile(q=0.5, interpolation='nearest')
varquan_05 = df[var00_name].quantile(q=0.75, interpolation='nearest')

# 모델 지정을 위한 for문 

for i in range(len(row_names)) :         

  # 모델 가져오기

  mov5_name = row_names[i] + "_mov5"
  updown_name = row_names[i] + "_updown"
  var00_name = row_names[i] + "_var00"
  
  # 최근 5일간 각 모델의 판단

  list1 = [-5,-4,-3,-2,-1]
  print("\n", row_names[i],"의 판단입니다.\n")

  for a in list1 :
    
    if df[mov5_name].values[a] >= movquan_01 :                                                                                                    # 현재수준 90% 이상
      print(df2[a],'기준 \'적극 투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')

    elif df[mov5_name].values[a] >= movquan_02 and df[updown_name].values[a] >= 0 :                                                               # 현재 수준 90% 미만 70% 이상 / 변동값 증가
      print(df2[a] ,'기준 \'투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')
    
    elif df[mov5_name].values[a] >= movquan_02 and df[updown_name].values[a] < 0 :                                                                # 현재 수준 90% 미만 70% 이상 / 변동값 감소
      print(df2[a] ,'기준 \'보수적 투자 권장\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a],2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] > 0 and df[var00_name].values[a] > varquan_04 :         # 현재 수준 70% 미만 40% 이상 / 변동값 증가 / 분산 50% 초과
      print(df2[a] ,'기준 \'투자 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] <= 0 and df[var00_name].values[a] > varquan_04 :        # 현재 수준 70% 미만 40% 이상 / 변동값 감소 / 분산 50% 초과
      print(df2[a] ,'기준 \'투자 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_02 > df[mov5_name].values[a] >= movquan_03 and df[updown_name].values[a] <= 0 and df[var00_name].values[a] < varquan_04 :        # 현재 수준 70% 미만 40% 이상 / 변동값 감소 / 분산 50% 미만
      print(df2[a] ,'기준 \'적정 시기가 아님\'을 추천하였습니다.')

    elif movquan_03 > df[mov5_name].values[a] and df[updown_name].values[a] > 0 and df[var00_name].values[a] > varquan_05 :                       # 현재 수준 40% 미만 / 변동값 증가 / 분산 75% 초과
      print(df2[a] ,'기준 \'오를 가능성이 있으니 관망\'을 추천하였습니다.'+ ' 적정 매수 가격은', round(df3[a]*.97,2),'입니다.')

    elif movquan_03 > df[mov5_name].values[a] and df[updown_name].values[a] <= 0 :                                                                # 현재 수준 40% 미만 / 변동값 감소 
      print(df2[a] ,'기준 \'투자를 권하지 않음\'을 추천하였습니다.')

    else :                                                         
      print(df2[a] ,'기준 \'투자를 권하지 않음\'을 추천하였습니다.')  



 ALPHA 의 판단입니다.

20201030 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 409.97 입니다.
20201102 기준 '투자를 권하지 않음'을 추천하였습니다.
20201103 기준 '오를 가능성이 있으니 관망'을 추천하였습니다. 적정 매수 가격은 393.68 입니다.
20201104 기준 '투자를 권하지 않음'을 추천하였습니다.
20201105 기준 '투자를 권하지 않음'을 추천하였습니다.

 BETA 의 판단입니다.

20201030 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 409.97 입니다.
20201102 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 406.02 입니다.
20201103 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 405.86 입니다.
20201104 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 408.85 입니다.
20201105 기준 '적극 투자 권장'을 추천하였습니다. 적정 매수 가격은 414.3 입니다.
